In [1]:
overpass_api_query = """
[out:json];
area[name="秋田県"]->.a;
(
  nwr["amenity"="school"](area.a);
);
out geom;
"""

import requests

overpass_api_res = requests.post("https://overpass-api.de/api/interpreter" , data = {"data": overpass_api_query})
overpass_api_json = overpass_api_res.json()

elements = []

for element in overpass_api_json['elements']:
    if 'geometry' in element:
        elements.append(element)

print(len(elements))

310


In [3]:
!pip install psycopg2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 8.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for psycopg2: filename=psycopg2-2.9.6-cp311-cp311-linux_x86_64.whl size=168908 sha256=33796d6b1bc9713de7f0bc020467a39d63458c6923ff143228b342317eaedec6
  Stored in directory: /home/jovyan/.cache/pip/wheels/a2/c5/3d/b8fd44767c125a321b8d6d7fbcce0181d2f2d1e34967c81365
Successfully built psycopg2


In [4]:
import psycopg2

# PostgreSQLの情報
host = 'postgis'
port = '5432'
user = 'postgres'
password = 'postgres'
database = 'sandbox'

# PostgreSQLに接続
conn = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)

# カーソルを作成
cursor = conn.cursor()

# テーブルを作成
cursor.execute('CREATE TABLE schools_akita_2023_07_19 (id BIGINT, name VARCHAR, name_en VARCHAR, geom GEOMETRY(Polygon, 4326));')

# 公園データを挿入
for element in elements:
    element_id = element['id']
    element_name = element['tags'].get('name', '').replace("'", "''")
    element_name_en = element['tags'].get('name:en', '').replace("'", "''")

    coordinates = element['geometry']

    # Polygonではないelementはスキップする
    if len(coordinates) < 3:
        continue

    # 座標データをPostGISの形式に変換
    polygon = 'POLYGON(('
    for coord in coordinates:
        lon = coord['lon']
        lat = coord['lat']
        polygon += f'{lon} {lat},'
    polygon = polygon.rstrip(',')  # 末尾のカンマを削除
    polygon += '))'

    # テーブルに挿入
    insert_query = f"INSERT INTO schools_akita_2023_07_19 (id, name, name_en, geom) VALUES ({element_id}, '{element_name}', '{element_name_en}', ST_SetSRID(ST_GeomFromText('{polygon}'), 4326));"
    print(insert_query)
    cursor.execute(insert_query)

# コミットして変更を確定
conn.commit()

# 接続を閉じる
conn.close()

INSERT INTO schools_akita_2023_07_19 (id, name, name_en, geom) VALUES (173221895, '秋田県立秋田南高等学校', '', ST_SetSRID(ST_GeomFromText('POLYGON((140.1216423 39.6921662,140.120182 39.6921571,140.1192354 39.6917212,140.1191824 39.6916182,140.119992 39.690717,140.1201248 39.6905709,140.1205658 39.6902286,140.1211548 39.6896542,140.1215577 39.6891882,140.1216789 39.688979,140.1219673 39.6884154,140.1229727 39.6882787,140.123282 39.6884176,140.123125 39.6887126,140.1234302 39.6888549,140.123405 39.6890089,140.1232852 39.6893394,140.1230084 39.6900935,140.1230932 39.690116,140.1231041 39.6901197,140.1230777 39.6901634,140.1229952 39.6902816,140.1228625 39.6904716,140.1226911 39.690712,140.1224975 39.6909933,140.1223287 39.69124,140.1221611 39.6914778,140.1219967 39.6917045,140.1217961 39.6919917,140.1216838 39.6921528,140.1216423 39.6921662))'), 4326));
INSERT INTO schools_akita_2023_07_19 (id, name, name_en, geom) VALUES (175946513, '大曲自動車学校', '', ST_SetSRID(ST_GeomFromText('POLYGON((140.4845386 3